In [17]:
import prefect
from prefect import task, Flow
import json
import boto3


@task(name="GetBooksIt")
def get_books_itbooks():
    lambda_client = boto3.client('lambda')

    response = lambda_client.invoke(
        FunctionName='trial-docker-dev-get-books-itbooks',
        InvocationType='RequestResponse',
    )
    
    assert response["StatusCode"] < 300
    
    payload = json.loads(response['Payload'].read())
    print(payload)
    return payload

@task(name='GetBooksNYTimes')
def get_books_nytimes():
    lambda_client = boto3.client('lambda')

    response = lambda_client.invoke(
        FunctionName='trial-docker-dev-get-books-nytimes',
        InvocationType='RequestResponse'
    )
    
    assert response["StatusCode"] < 300

    payload = json.loads(response['Payload'].read())
    print(payload)
    return payload
    
@task(name='InsertBooks')
def insert_books(books, source):
    lambda_client = boto3.client('lambda')

    response = lambda_client.invoke(
        FunctionName='trial-docker-dev-insert-book',
        InvocationType='RequestResponse',
        Payload = json.dumps({'source':source, 'books': books})
        # Payload = f'{{"books":{json.dumps(books)}}}'
    )
    print('pepe')
    assert response["StatusCode"] < 300

@task(name='UpdateTable')
def update_stats_table():
    lambda_client = boto3.client('lambda')

    response = lambda_client.invoke(
        FunctionName='trial-docker-dev-update-table',
        InvocationType='RequestResponse',
    )
    
    assert response["StatusCode"] < 300

with Flow('ETL') as flow:
    it_books = get_books_itbooks()
    nytimes_books = get_books_nytimes()

    # insert_it_books = insert_books.map(it_books, upstream_tasks=[it_books])
    # insert_nytimes_books = insert_books.map(nytimes_books, upstream_tasks=[nytimes_books])
    insert_it_books = insert_books(it_books,source='itbooks-api', upstream_tasks=[it_books])
    insert_nytimes_books = insert_books(nytimes_books, source='nytimes-api', upstream_tasks=[nytimes_books])
    update_stats_table(upstream_tasks=[insert_it_books, insert_nytimes_books])


# @task
# def etl_bronze(bucket_name, prizes_path, laureates_path, aws_key, aws_secret, aws_token, aws_region):
#     lambda_client = boto3.client('lambda', aws_access_key_id=aws_key, aws_secret_access_key=aws_secret, aws_session_token=aws_token, region_name=aws_region)
#     response = lambda_client.invoke(
#         FunctionName='bronzeETL',
#         InvocationType='RequestResponse',
#         Payload = f'{{"bucketName": "{bucket_name}", "prizesPath": "{prizes_path}", "laureatePath": "{laureates_path}"}}'
#     )
#     assert response["StatusCode"] < 300

# @task
# def etl_silver(bronze_path, aws_key, aws_secret, aws_token, aws_region):    
#     lambda_client = boto3.client('lambda', aws_access_key_id=aws_key, aws_secret_access_key=aws_secret, aws_session_token=aws_token, region_name=aws_region)
#     response = lambda_client.invoke(
#         FunctionName='silverETL',
#         InvocationType='RequestResponse',
#         Payload = f'{{"bronzePath": "{bronze_path}"}}'
#     )
#     assert response["StatusCode"] < 300


# with Flow('ETL') as flow:
#     aws_key = Parameter("aws_access_key_id")
#     aws_secret = Parameter("aws_secret_access_key")
#     aws_token = Parameter("aws_session_token")
#     aws_region = Parameter("aws_region")
#     for category in ['che', 'eco', 'lit', 'pea', 'phy', 'med']:
#         laureates = extract_nobel_prizes(category, aws_key, aws_secret, aws_token, aws_region)
#         x1 = extract_laureate.map(laureates, unmapped(aws_key), unmapped(aws_secret), unmapped(aws_token), unmapped(aws_region), upstream_tasks=[laureates])
#     x2 = etl_bronze("capstone-13", "s3://capstone-13/raw/nobelPrizes/", "s3://capstone-13/raw/laureates/", aws_key, aws_secret, aws_token, aws_region, upstream_tasks=[x1])
#     etl_silver("s3://capstone-13/bronze/bronze_laureates.parquet", aws_key, aws_secret, aws_token, aws_region, upstream_tasks=[x2])


In [3]:
flow.visualize()

ExecutableNotFound: failed to execute PosixPath('dot'), make sure the Graphviz executables are on your systems' PATH

In [18]:
flow.run()

[2022-08-22 18:47:03+0200] INFO - prefect.FlowRunner | Beginning Flow run for 'ETL'
[2022-08-22 18:47:03+0200] INFO - prefect.TaskRunner | Task 'GetBooksIt': Starting task run...
[]
[2022-08-22 18:47:06+0200] INFO - prefect.TaskRunner | Task 'GetBooksIt': Finished task run for task with final state: 'Success'
[2022-08-22 18:47:06+0200] INFO - prefect.TaskRunner | Task 'InsertBooks': Starting task run...
pepe
[2022-08-22 18:47:07+0200] INFO - prefect.TaskRunner | Task 'InsertBooks': Finished task run for task with final state: 'Success'
[2022-08-22 18:47:08+0200] INFO - prefect.TaskRunner | Task 'GetBooksNYTimes': Starting task run...
['clean/debbie_macomber/9780553391862.json', 'clean/james_patterson_and_david_ellis/9781455589913.json', 'clean/j_d_robb/9781101987995.json', 'clean/w_bruce_cameron/9780765388100.json', 'clean/catherine_anderson/9780451488022.json', 'clean/fern_michaels/9781420140651.json', 'clean/jonathan_kellerman/9780345541420.json', 'clean/paula_hawkins/9780735212169.j

<Success: "All reference tasks succeeded.">